### NSP Cases



Here are the libraries / configurations used.

In [1]:
from abm_scheduling import Schedule as Schedule
from abm_scheduling import Nurse as Nurse

In [2]:
#%% Defined numbers
# Mo 1,2,3, Tu 1,2,3, We...
matrix_nurses_needed = [5,8,5, 4,7,4, 4,7,4, 4,7,5, 5,7,7, 6,7,8, 7,5,5]
schedule_random = Schedule(matrix_nurses_needed=matrix_nurses_needed, is_random=False)
schedule_random.print_schedule()

Week's Schedule
+---------+----------+----------+----------+----------+----------+----------+----------+
|         | Mo       | Tu       | We       | Th       | Fr       | Sa       | So       |
+---------+----------+----------+----------+----------+----------+----------+----------+
| shift 1 | need: 5  | need: 4  | need: 4  | need: 4  | need: 5  | need: 6  | need: 7  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 2 | need: 8  | need: 7  | need: 7  | need: 7  | need: 7  | need: 7  | need: 5  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
| shift 3 | need: 5  | need: 4  | need: 4  | need: 5  | need: 7  | need: 8  | need: 5  |
|         | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  | nurses:  |
+---------+----------+----------+----------+----------+----------+----------+----------+


In [ ]:
# --example schedules--
schedule_fixed = Schedule(num_nurses_needed=3)
schedule_fixed.print_schedule()

In [ ]:
schedule_random = Schedule(num_nurses_needed=3, is_random=True)
schedule_random.print_schedule()

In [ ]:
# --example nurses--
# full time, no weekends
nurse = Nurse(id_name=0)
nurse.generate_shift_preferences(degree_of_agent_availability=1, works_weekends=False)
nurse.print_shift_preferences()

In [ ]:
# full time, weekends
nurse = Nurse(id_name=1)
nurse.generate_shift_preferences(degree_of_agent_availability=1, works_weekends=True)
nurse.print_shift_preferences()

In [ ]:
# part time, weekends
nurse = Nurse(id_name=3)
nurse.generate_shift_preferences(degree_of_agent_availability=0.5, works_weekends=True)
nurse.print_shift_preferences()

In [3]:
# part time, no weekends
nurse = Nurse(id_name=2)
nurse.generate_shift_preferences(degree_of_agent_availability=0.78, works_weekends=False)
nurse.print_shift_preferences()

Nurse 2's Preferences
+---------+----+----+----+----+----+----+----+
|         | Mo | Tu | We | Th | Fr | Sa | So |
+---------+----+----+----+----+----+----+----+
| shift 1 | x  | x  | x  |    | x  |    |    |
| shift 2 | x  |    | x  | x  | x  |    |    |
| shift 3 |    | x  | x  | x  |    |    |    |
+---------+----+----+----+----+----+----+----+


In [ ]:
# --example generating nurses--
# generate 4 nurses that work 50% and on weekends
nurses = []
num_nurses = 4
for n in range(num_nurses):
    nurse = Nurse(id_name=n)
    nurse.generate_shift_preferences(degree_of_agent_availability=0.5, works_weekends=True)
    nurses.append(nurse)
    nurse.print_shift_preferences()

In [ ]:
# -- example assign nurse to shift --
nurses[0].print_shift_preferences()

In [ ]:
schedule = Schedule(num_nurses_needed=3)
candidate_schedule = copy.deepcopy(schedule)
best_schedule = copy.deepcopy(schedule)
best_utility = 0
nurses = generate_nurses(10, 0.5, True)
p_to_accept_negative_change = .001
# show what all shift preferences look like in the schedule
hypothetical_max_schedule = copy.deepcopy(schedule)
for nurse in nurses:
    for shift in nurse.shift_preferences:
        hypothetical_max_schedule.add_nurse_to_shift(nurse, shift, False)
        
schedule.print_schedule()
hypothetical_max_schedule.print_schedule()
print('Crude hypothetical shift coverage:', hypothetical_max_schedule.get_shift_coverage())

utility_each_timestep = []
shift_coverage_each_timestep = []
# timestep is for each nurse, so total timesteps = x * num_nurses where x is range(x)
for timestep in range(10000):
    for nurse in nurses:
        schedule_utility = schedule.get_utility()
        utility_each_timestep.append(schedule_utility)
        shift_coverage_each_timestep.append(schedule.get_shift_coverage())
        # keep track of best utility
        if schedule_utility > best_utility:
            best_schedule.schedule = copy.deepcopy(schedule.schedule)
            best_utility = schedule_utility
        rnd_i = rnd.randint(len(nurse.shift_preferences))
        rnd_shift_pref = nurse.shift_preferences[rnd_i]
        was_in_shift = rnd_shift_pref in nurse.shifts
        # try adding/removing shift depending on whether the shift is assigned
        if not was_in_shift:
            candidate_schedule.add_nurse_to_shift(nurse, rnd_shift_pref, False)
        else:
            candidate_schedule.remove_nurse_from_shift(nurse, rnd_shift_pref, False)
        # if the change was better or randomly accept negative change, apply change to schedule
        if (candidate_schedule.get_utility() > schedule_utility) or (rnd.random_sample() < p_to_accept_negative_change):
            if not was_in_shift:
                schedule.add_nurse_to_shift(nurse, rnd_shift_pref, True)
            else:
                schedule.remove_nurse_from_shift(nurse, rnd_shift_pref, True)
        # if the change was worse, undo the change to the candidate schedule
        else:
            if not was_in_shift:
                candidate_schedule.remove_nurse_from_shift(nurse, rnd_shift_pref, False)
            else:
                candidate_schedule.add_nurse_to_shift(nurse, rnd_shift_pref, False)

best_schedule.print_schedule()
print('Solution shift coverage:',best_schedule.get_shift_coverage())
print('Solution utility', best_schedule.get_utility())

plt.figure()
plt.plot(utility_each_timestep, label = "utility")
plt.title("Schedule utility of each timestep")
plt.xlabel("timestep")
plt.ylabel("utility")
plt.legend()
plt.show()

plt.figure()
plt.plot(shift_coverage_each_timestep, label = "shift coverage")
plt.title("Shift coverage of each timestep")
plt.xlabel("timestep")
plt.ylabel("shift coverage")
plt.legend()
plt.show()